In [ ]:
import numpy as np
import pandas as pd
#import xmltodict
import glob
import re
from os.path import basename, splitext

In [ ]:
data_dir = glob.glob('../../Data/Training/*')
df = {}

In [ ]:
for genre_file in data_dir:
    with open(genre_file) as f:
        tmp = f.read().split('</doc>')
        name = splitext(basename(genre_file))[0]
        df[name] = tmp
        del tmp

In [ ]:
tmp = []
for name, corpus in df.items():
    i = 0
    for item in corpus:
        m = re.search('<doc id="[0-9]+" genre="[a-zA-Z]+" gender="[M|F]">', item)
        if m is not None:
            i += 1
            tmp.append((m.group(0), item))
    assert i==len(corpus)-1
del df
df = pd.DataFrame(tmp)


In [ ]:
df.rename(columns={0:'meta', 1:'text'}, inplace=True)
df['gender'] = df.iloc[:,0].str.extract(r'(M|F)')
df['genre'] = df.iloc[:,0].str.extract(r'(twitter|youtube|diary|journalism|children)')
df['docid'] = df.iloc[:,0].str.extract(r'([0-9]+)')
df.sample(5)

In [ ]:
for idx, row in df.iterrows():
    row.text = row.text.replace(row.meta, '').strip()

In [ ]:
df.sample(3)

In [ ]:
df.to_json('../../Data/gxg.json', orient='records', lines=True)

In [7]:
%%time
import numpy as np
import pandas as pd
import sys
# Add the ptdraft folder path to the sys.path list
sys.path.append('../')
from labov import datasets, classifier, evaluation
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import shuffle
import spacy
nlp = spacy.load('it')
from collections import Counter

CPU times: user 174 ms, sys: 44 ms, total: 218 ms
Wall time: 217 ms


In [2]:
df = pd.read_json('../../Data/gxg.json', lines=True)
df.head()

,docid,gender,genre,meta,text
0,1,M,children,"<doc id=""1"" genre=""children"" gender=""M"">",… Questa estate mi sono divertito molto perché...
1,2,M,children,"<doc id=""2"" genre=""children"" gender=""M"">",Benvenuti a “TGDVD” i marò sono stati liberati...
2,3,M,children,"<doc id=""3"" genre=""children"" gender=""M"">","Roma, 9 Giungo 2012\nCaro diario,\nOggi vorrei..."
3,4,M,children,"<doc id=""4"" genre=""children"" gender=""M"">","Roma 9 Giugno 2012\nCaro diario,\nil mio ultim..."
4,5,M,children,"<doc id=""5"" genre=""children"" gender=""M"">","Io non so bene come definirmi, ma secondo me s..."


## In-genre

In [4]:
for model in [classifier.random, classifier.ngram, classifier.words, classifier.chars]:
    for n, g in df.groupby('genre'):
        print(n)
        scores = cross_val_score(model, g.text, g.gender, cv=10)
        print(scores.mean(), scores.std())
    print('----')

children
0.5 0.05000000000000002
diary
0.45 0.0
journalism
0.45 0.0
twitter
0.48233333333333334 0.01686877457184
youtube
0.49727272727272726 0.021723278446088228
----
children
0.5249999999999999 0.08139410298049853
diary
0.7050000000000001 0.08789197915623471
journalism
0.615 0.11629703349613006
twitter
0.7366666666666667 0.015811388300841903
youtube
0.6154545454545455 0.02179449471770338
----
children
0.55 0.09746794344808965
diary
0.73 0.08717797887081347
journalism
0.655 0.115
twitter
0.7278333333333333 0.015074813431681329
youtube
0.6034090909090908 0.020811283096729794
----
children
0.525 0.125
diary
0.68 0.11874342087037916
journalism
0.54 0.13
twitter
0.7416666666666667 0.014337208778404381
youtube
0.6213636363636363 0.02125212677738357
----


## Cross-genre

In [19]:
for model in [classifier.random, classifier.words, classifier.chars, classifier.ngram]:
    print('-----')
    for genre in set(df.genre):
        test = df[df.genre == genre]
        train = df[df.genre != genre]
        m = model.fit(train.text, train.gender)
        y_pred = m.predict(test.text)
        print(genre, accuracy_score(y_pred, test.gender))
    print('---')

-----
youtube 0.5059090909090909
children 0.45
diary 0.56
journalism 0.56
twitter 0.49233333333333335
---
-----
youtube 0.5306818181818181
children 0.585
diary 0.64
journalism 0.55
twitter 0.5153333333333333
---
-----
youtube 0.5561363636363637
children 0.595
diary 0.675
journalism 0.52
twitter 0.5401666666666667
---
-----
youtube 0.5563636363636364
children 0.6
diary 0.625
journalism 0.56
twitter 0.5171666666666667
---


## Bleaching

In [8]:
wordCount = Counter(df.text.str.cat(sep='\n').split(' '))

In [10]:
def shaper(word):
    out = []
    for char in word.text:
        if char.isupper():
            if char in 'AEIOUY':
                out.append('V')
            elif char in 'BCDFGHJKLMNPQRSTVXZ':
                out.append('C')
            else:
                out.append(char)
        elif char.islower():
            if char in 'aeiouyèéàòù':
                out.append('v')
            elif char in 'BCDFGHJKLMNPQRSTVXZ'.lower():
                out.append('c')
            else:
                out.append(char)
        else:
            out.append(char)
    return ''.join(out)

def bleach(text, word_dic=None):
    bleached = ' '.join([' '.join((shaper(w), 
                                   str(0)+str(len(w)),
                                   str(w.is_alpha),
                                   str(word_dic[w.text]))) for w in text])
    return bleached

bleach(nlp('this TeXt!!!'), wordCount)

'ccvc 04 True 0 CvCc 04 True 0 ! 01 False 135 ! 01 False 135 ! 01 False 135'

In [11]:
textit = (row.text for row in df.itertuples())
df['spacy'] = [doc for doc in nlp.pipe(textit,n_threads=4, batch_size=1000)]
df['bleached'] = df.spacy.apply(lambda x: bleach(x, wordCount))

In [12]:
df.sample(3)

,docid,gender,genre,meta,text,spacy,bleached
806,9499,F,youtube,"<doc id=""9499"" genre=""youtube"" gender=""F"">",Favij io ti considero mates,"(Favij, io, ti, considero, mates)",Cvcvc 05 True 1 vv 02 True 784 cv 02 True 665 ...
9644,7911,F,twitter,"<doc id=""7911"" genre=""twitter"" gender=""F"">","Dunque , devo tradurre una prezzo di racconto ...","(Dunque, ,, devo, tradurre, una, prezzo, di, r...","Cvccvv 06 True 27 , 01 False 466 cvcv 04 True ..."
6692,7768,F,twitter,"<doc id=""7768"" genre=""twitter"" gender=""F"">","Quindi #TempaRossa , per #Renzi serve per espo...","(Quindi, #, TempaRossa, ,, per, #, Renzi, serv...",Cvvccv 06 True 25 # 01 False 0 CvccvCvccv 010 ...


## In-genre

In [21]:
for n, g in df.groupby('genre'):
    print(n)
    scores = cross_val_score(classifier.ngram, g.bleached, g.gender, cv=10)
    print(scores.mean(), scores.std())
    print('----')

children
0.5349999999999999 0.07088723439378912
----
diary
0.6799999999999999 0.10770329614269007
----
journalism
0.545 0.11056672193747988
----
twitter
0.6975 0.014553540997144149
----
youtube
0.5945454545454545 0.02355756094518171
----


## Cross-genre

In [20]:
for genre in set(df.genre):
    test = df[df.genre == genre]
    train = df[df.genre != genre]
    m = model.fit(train.bleached, train.gender)
    y_pred = m.predict(test.bleached)
    print(genre, accuracy_score(y_pred, test.gender))
    print('---')

youtube 0.5297727272727273
---
children 0.535
---
diary 0.585
---
journalism 0.53
---
twitter 0.49933333333333335
---
